In [ ]:
#Import packages

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
import math
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn import linear_model, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.cross_validation import cross_val_predict
from sklearn.svm.libsvm import cross_validation
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import data
postcodes2017 = pd.read_excel("postcodes2017.xlsx")

# Adjust missing data for most recent homes
for i in postcodes2017.index:
    if postcodes2017.loc[i, 'WON_1524,N,10,0'] == -99997:
        postcodes2017.loc[i, 'WON_1524,N,10,0'] = 0

# Select relevant columns        
columns = ['PC4,N,5,0', 'INWONER,N,10,0', 'MAN,N,10,0', 'VROUW,N,10,0',
       'INW_014,N,10,0', 'INW_1524,N,10,0', 'INW_2544,N,10,0',
       'INW_4564,N,10,0', 'INW_65PL,N,10,0', 'GEBOORTE,N,10,0',
       'P_NL_ACHTG,N,10,0', 'P_WE_MIG_A,N,10,0', 'P_NW_MIG_A,N,10,0',
       'AANTAL_HH,N,10,0', 'TOTHH_EENP,N,10,0', 'TOTHH_MPZK,N,10,0',
       'HH_EENOUD,N,10,0', 'HH_TWEEOUD,N,10,0', 'GEM_HH_GR,N,19,11',
       'WONING,N,10,0', 'WONVOOR45,N,10,0', 'WON_4564,N,10,0',
       'WON_6574,N,10,0', 'WON_7584,N,10,0', 'WON_8594,N,10,0',
       'WON_9504,N,10,0', 'WON_0514,N,10,0', 'WON_1524,N,10,0',
       'WON_MRGEZ,N,10,0', 'P_HUURWON,N,10,0', 'P_KOOPWON,N,10,0',
       'WON_HCORP,N,10,0', 'WON_NBEW,N,10,0',
       'G_GAS_WON,N,10,0', 'G_ELEK_WON,N,10,0', 'UITKMINAOW,N,10,0', 'STED,N,10,0']

postcodes2017 = postcodes2017[columns]

# Set index to postal code
postcodes2017 = postcodes2017.set_index("PC4,N,5,0")

In [ ]:
# Create data backup
data = postcodes2017.copy()

# Change missing data to NaN
for i in data.index:
    for j in data.columns:
        if data.loc[i, j] == -99997:
            data.loc[i,j] = float('NaN')

# Create percentage columns
data['PERC_EENP'] = data['TOTHH_EENP,N,10,0'] / data['AANTAL_HH,N,10,0'] * 100
data['PERC_MPZK'] = data['TOTHH_MPZK,N,10,0'] / data['AANTAL_HH,N,10,0'] * 100
data['PERC_EENOUD'] = data['HH_EENOUD,N,10,0'] / data['AANTAL_HH,N,10,0'] * 100
data['PERC_TWEEOUD'] = data['HH_TWEEOUD,N,10,0'] / data['AANTAL_HH,N,10,0'] * 100

data['PERC_MAN'] = data['MAN,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_VROUW'] = 100 - data['PERC_MAN']
data['PERC_014'] = data['INW_014,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_1524'] = data['INW_1524,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_2544'] = data['INW_2544,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_4564'] = data['INW_4564,N,10,0'] / data['INWONER,N,10,0'] * 100
data['PERC_65PL'] = data['INW_65PL,N,10,0'] / data['INWONER,N,10,0'] * 100

data['TOTAAL_WON'] = data['WONVOOR45,N,10,0'] + data['WON_4564,N,10,0'] + data['WON_6574,N,10,0'] + data['WON_7584,N,10,0'] + data['WON_8594,N,10,0'] + data['WON_9504,N,10,0'] + data['WON_0514,N,10,0'] + data['WON_1524,N,10,0']

data['PERC_WON_045'] = data['WONVOOR45,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_4564'] = data['WON_4564,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_6574'] = data['WON_6574,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_7584'] = data['WON_7584,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_8594'] = data['WON_8594,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_9504'] = data['WON_9504,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_0514'] = data['WON_0514,N,10,0'] / data['TOTAAL_WON'] * 100
data['PERC_WON_1524'] = data['WON_1524,N,10,0'] / data['TOTAAL_WON'] * 100

# Select relevant columns
columns = ['PERC_MAN', 'PERC_014', 'PERC_1524', 'PERC_2544',
       'PERC_4564', 'PERC_65PL',
       'P_NL_ACHTG,N,10,0', 'P_WE_MIG_A,N,10,0', 'P_NW_MIG_A,N,10,0', 'PERC_EENP', 'PERC_MPZK',
       'PERC_EENOUD', 'PERC_TWEEOUD', 'PERC_WON_045', 'PERC_WON_4564',
       'PERC_WON_6574', 'PERC_WON_7584', 'PERC_WON_8594',
       'PERC_WON_9504', 'PERC_WON_0514', 'PERC_WON_1524', 'G_GAS_WON,N,10,0', 'G_ELEK_WON,N,10,0', 'WON_MRGEZ,N,10,0', 'STED,N,10,0', 'GEM_HH_GR,N,19,11']

data = data[columns]

# Remove missing data
data = data.dropna()

# Create data backup
data2 = data.copy()

# Scale data
X = X.values
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(x_scaled)
X.columns = columns
X.index = ind
data = X.copy()

In [ ]:
def analyse7(data, iv, lasso, energy, data2):
    # This function runs an elastic net model for the data set. It takes parameters on whether to include
    # interaction variables and what energy type to run the analysis for
    
    X = data.copy()
    y = data2[energy]
    
    # Select relevant columns
    columns = ['PERC_MAN', 'PERC_1524', 'PERC_2544',
       'PERC_4564', 'PERC_65PL',
       'P_NL_ACHTG,N,10,0', 'P_WE_MIG_A,N,10,0', 'PERC_MPZK',
       'PERC_EENOUD', 'PERC_TWEEOUD', 'PERC_WON_4564',
       'PERC_WON_6574', 'PERC_WON_7584', 'PERC_WON_8594',
       'PERC_WON_9504', 'PERC_WON_0514', 'PERC_WON_1524',]
    
    X = X[columns]
    
    # Create interaction variables
    if iv == 1:
        vis = []
        fish = []

        for i in columns:
            for j in columns:
                if j not in fish:
                    vis.append([i,j])
            fish.append(i)
            
        columns2 = columns.copy()

        for i in vis:
            columns2.append(i)
            
        for i in columns2:
            if len(i) == 2:
                X[i[0] + ' x ' + i[1]] = X[i[0]] * X[i[1]]
            
#         X = X.iloc[:, 17:]
    
    # Manually create trainig and test split, due to bug in Scikit package

    list1 = set(X.index)

    train = sample(list1, int(0.7 * len(list1)))

    X_train = X.loc[train, :]
    y_train = y[train]

    train = set(train)

    test = list1 - train

    X_test = X.loc[test, :]
    test = list(test)
    y_test = y[test]
    
    # Run elastic net for various lambda parameters
    if True:
        values = [0, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 20, 30]
        scores = []
        scores2 = []

        if True:
            for i in values:
                #Lasso om variabelen te bepalen
                clf = linear_model.ElasticNet(alpha=i, l1_ratio = lasso)

                clf.fit(X_train,y_train)

                preds = list(clf.predict(X_test))
                trues = list(y_test)

                errors = [a_i - b_i for a_i, b_i in zip(preds, trues)]

                rmse = [math.sqrt(x**2) for x in errors]

                scores.append(sum(rmse)/len(rmse))
                
                print(sum(abs(clf.coef_))/len(clf.coef_))
                print(len(clf.coef_[clf.coef_ != 0]))
        
                # Manually calculate R-squared
                coef = clf.coef_
                p = numpy.poly1d(coef)
                yhat = clf.predict(X_train)

                ybar = numpy.sum(y_train)/len(y_train)          # or sum(y)/len(y)
                ssreg = numpy.sum((yhat - ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
                sstot = numpy.sum((y_train - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
                print(ssreg / sstot)
        
                print('--------------------------')
            
                coef = clf.coef_
                
                # Manually create the standard errors of the coefficients
                MSE = np.mean((y_train - clf.predict(X_train).T)**2)
                var_est = MSE * np.diag(np.linalg.pinv(np.dot(X_train.T,X_train)))
                SE_est = np.sqrt(var_est)
                
                temp = pd.DataFrame()
                temp['Coef'] = coef
                
                # Calculate t-values
                tvalues = coef / SE_est
                
                # Calculate p-values
                pvalues = []
                for k in tvalues:
                    pvalues.append(scipy.stats.t.sf(abs(k), df=0.7*len(data)-len(coef)))
                    
                temp['P-value'] = pvalues
                
                length = len(temp[(temp['Coef'] != 0) & (temp['P-value'] < 0.01)])
            
                scores2.append([i, sum(abs(clf.coef_)), sum(rmse)/len(rmse), length, ssreg / sstot])
            
            # Rerun the model for the best performing lambda parameter
            max_value = min(scores)
            max_index = scores.index(max_value)

            clf = linear_model.ElasticNet(alpha=values[max_index], l1_ratio = lasso)

            clf.fit(X_train,y_train)

            coef = clf.coef_
            
            MSE = np.mean((y_train - clf.predict(X_train).T)**2)
            var_est = MSE * np.diag(np.linalg.pinv(np.dot(X_train.T,X_train)))
            SE_est = np.sqrt(var_est)
            
    return [coef, SE_est, X, scores2]

In [ ]:
coef, se, X, scores2 = analyse7(data, 0, 0.3, 'G_GAS_WON,N,10,0', data2)
tvalues = coef / se

#find p-value
pvalues = []
for i in tvalues:
    pvalues.append(scipy.stats.t.sf(abs(i), df=0.7*len(data)-len(coef)))

results = pd.DataFrame(index=X.columns)

results['Coefficient'] = coef
results['Standard Error'] = se
results['P-value'] = pvalues

pd.DataFrame(scores2)

In [ ]:
def analyse6(data, iv, lasso, energy, data2, alpha):
    
    X = data.copy()
    y = data2[energy]
    
    columns = ['PERC_MAN', 'PERC_1524', 'PERC_2544',
       'PERC_4564', 'PERC_65PL',
       'P_NL_ACHTG,N,10,0', 'P_WE_MIG_A,N,10,0', 'PERC_MPZK',
       'PERC_EENOUD', 'PERC_TWEEOUD', 'PERC_WON_4564',
       'PERC_WON_6574', 'PERC_WON_7584', 'PERC_WON_8594',
       'PERC_WON_9504', 'PERC_WON_0514', 'PERC_WON_1524',]
    
    X = X[columns]
    
    if iv == 1:
        vis = []
        fish = []

        for i in columns:
            for j in columns:
                if j not in fish:
                    vis.append([i,j])
            fish.append(i)
            
        columns2 = columns.copy()

        for i in vis:
            columns2.append(i)
            
        for i in columns2:
            if len(i) == 2:
                X[i[0] + ' x ' + i[1]] = X[i[0]] * X[i[1]]
            
#         X = X.iloc[:, 17:]
        
    cols = ['PERC_MAN', 'PERC_2544', 'PERC_4564', 'P_NL_ACHTG,N,10,0',
       'PERC_EENOUD', 'PERC_TWEEOUD', 'PERC_WON_6574', 'PERC_WON_7584',
       'PERC_WON_8594', 'PERC_WON_9504', 'PERC_WON_0514',
       'PERC_MAN x PERC_EENOUD', 'PERC_MAN x PERC_WON_8594',
       'P_NL_ACHTG,N,10,0 x PERC_WON_7584',
       'P_NL_ACHTG,N,10,0 x PERC_WON_0514', 'PERC_WON_0514 x PERC_WON_1524']
    
    X = X[cols]

    # Prints list of random items of given length 
    list1 = set(X.index)

    train = sample(list1, int(0.7 * len(list1)))

    X_train = X.loc[train, :]
    y_train = y[train]

    train = set(train)

    test = list1 - train

    X_test = X.loc[test, :]
    test = list(test)
    y_test = y[test]
        
    if True:
        values = [10**-3]
        scores = []
        scores2 = []

        if True:
            for i in values:
                #Lasso om variabelen te bepalen
                from sklearn import linear_model
                clf = linear_model.ElasticNet(alpha=alpha, l1_ratio = lasso, fit_intercept=True)

                clf.fit(X_train,y_train)

                preds = list(clf.predict(X_test))
                trues = list(y_test)

                errors = [a_i - b_i for a_i, b_i in zip(preds, trues)]

                rmse = [math.sqrt(x**2) for x in errors]
                
                scores2.append(sum(rmse)/len(rmse))
                print(sum(rmse)/len(rmse))

                scores.append(sum(rmse)/len(rmse))
                
                coef = clf.coef_
                
                # r-squared
                coef = clf.coef_
                p = numpy.poly1d(coef)
                # fit values, and mean
                yhat = clf.predict(X_train)

                ybar = numpy.sum(y_train)/len(y_train)          # or sum(y)/len(y)
                ssreg = numpy.sum((yhat - ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
                sstot = numpy.sum((y_train - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
                print(ssreg / sstot)
                
                print(sum(clf.coef_)/len(clf.coef_))
                print(len(clf.coef_[clf.coef_ != 0]))
                print('--------------------------')

            MSE = np.mean((y_train - clf.predict(X_train).T)**2)
            var_est = MSE * np.diag(np.linalg.pinv(np.dot(X_train.T,X_train)))
            SE_est = np.sqrt(var_est)
            
            print(clf.intercept_)
            
            
            
    return [coef, SE_est, X, scores2]

In [ ]:
coef, se, X, scores2 = analyse6(data, 1, 0.3, 'G_GAS_WON,N,10,0', data2, 0.1)
tvalues = coef / se

# Find p-value
pvalues = []
for i in tvalues:
    pvalues.append(scipy.stats.t.sf(abs(i), df=0.7*len(data)-len(coef)))

results = pd.DataFrame(index=X.columns)

results['Coefficient'] = coef
results['Standard Error'] = se
results['P-value'] = pvalues

clean_results = results[results['Coefficient'] != 0]
clean_results

In [ ]:
res = pd.DataFrame(scores2)

fig, ax1 = plt.subplots()

plt.xscale('log')

color = 'tab:red'
ax1.set_xlabel('Alpha')
ax1.set_ylabel('Sum of coefficient', color=color)
ax1.plot(res[0], res[1], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:green'
ax2.set_ylabel('RMSE', color=color)  # we already handled the x-label with ax1
ax2.plot(res[0], res[3], color=color)
ax2.tick_params(axis='y', labelcolor=color)

color = 'tab:blue'
ax2.set_ylabel('RMSE', color=color)  # we already handled the x-label with ax1
ax2.plot(res[0], res[2], color=color)
ax2.tick_params(axis='y', labelcolor=color)


fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()